# **Distribución de propiedades por segmento**

In [11]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

In [22]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and  'queretaro' in nombre and 'financiamientos' not in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro',
 'df_junio_2023_queretaro',
 'df_poblacion_queretaro']

## 2023

In [23]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and  'df_junio_2023_queretaro' in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_junio_2023_queretaro']

In [24]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','Categoria':'categoria','Banos_Total':'baño_total','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido',}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [25]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['categoria','precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [26]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

In [27]:
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])
conteo_total_df

,categoria,conteo_total
0,S,2622
1,B1,728
2,B2,674
3,B3,666
4,A2,603
5,A3,572
6,C1,472
7,A1,449
8,C2,434
9,C3,257


In [28]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,2622,S,1
1,728,B,1
2,674,B,2
3,666,B,3
4,603,A,2
5,572,A,3
6,472,C,1
7,449,A,1
8,434,C,2
9,257,C,3


In [19]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  7861


,segmento,total_por_segmento,porcentaje_del_total
0,A,1624,20.658949
1,B,2068,26.307086
2,C,1163,14.794555
3,D,357,4.541407
4,E,27,0.343468
5,S,2622,33.354535


In [20]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

fig.show()

In [37]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2023', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2024

In [38]:
# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "queretaro" in nombre 
    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_queretaro',
 'df_alfa_agosto_2024_queretaro',
 'df_alfa_marzo_2024_queretaro',
 'df_alfa_mayo_2024_queretaro']

In [39]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    segment_df = globals()[nombre_df][['Categoria','Precio']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

In [40]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
from collections import Counter

# Crear un diccionario para almacenar el conteo total de cada categoría
conteo_total = Counter()

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Verificar que la columna 'Precio' exista en el DataFrame
    if 'Precio' in df.columns:
        # Asignar la categoría a cada registro basado en su precio
        df['categoria'] = df['Precio'].apply(asignar_categoria)
        
        # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
        conteo_actual = Counter(df['categoria'])
        
        # Sumar el conteo actual al conteo total
        conteo_total.update(conteo_actual)

# Convertir el contador a un DataFrame
conteo_total_df = pd.DataFrame(conteo_total.items(), columns=['categoria', 'conteo_total'])

# Mostrar el DataFrame final con el conteo total por categoría
conteo_total_df


C:\Users\yoe11\AppData\Local\Temp\ipykernel_15556\547108086.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15556\547108086.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_15556\547108086.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,categoria,conteo_total
0,B1,1312
1,S2,4506
2,B2,1912
3,E1,128
4,E2,671
5,None,2440
6,E3,1107
7,D1,485
8,D2,795
9,ELITE,927


In [41]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Añadir una nueva fila como un DataFrame
#nueva_fila_df = pd.DataFrame({'conteo_total': [0], 'segmento': ['E'], 'clase': ['3']})
# Concatenar el DataFrame original con la nueva fila
conteo_total_dfs = pd.concat([conteo_total_df, nueva_fila_df], ignore_index=True)
conteo_total_dfs

,conteo_total,segmento,clase
0,1312,B,1
1,4506,S,2
2,1912,B,2
3,128,E,1
4,671,E,2
5,2440,None,1
6,1107,E,3
7,485,D,1
8,795,D,2
9,927,E,1


In [42]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_dfs.groupby('segmento')['conteo_total'].sum() 
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  36171


,segmento,total_por_segmento,porcentaje_del_total
0,A,4473,12.366260
1,B,4665,12.897072
2,C,2394,6.618562
3,D,1954,5.402118
4,E,2833,7.832241
5,L,1760,4.865776
6,S,18092,50.017970


In [43]:
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

fig = go.Figure(data=[go.Pie(labels=segmentos, values=porcentajes_totales, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(
    yaxis=dict(
            # title='Rango de Edad', 
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        ),
        legend=dict(
            font_size=10,
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
            ),
        
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
    # pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

fig.show()

In [44]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

# Asegurar que la columna de segmentos tenga el orden deseado
resultados['segmento'] = pd.Categorical(resultados['segmento'], categories=orden_segmentos, ordered=True)

# Reordenar el DataFrame según el orden de los segmentos
resultados = resultados.sort_values('segmento', ascending=False)
segmentos = resultados['segmento']
porcentajes_totales = resultados['porcentaje_del_total']

# Definir los colores por clase
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in segmentos.values else '#A9A9A9' 
                     for i, segmento in enumerate(orden_segmentos)}

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes_totales[segmentos == segmento].sum() if segmento in segmentos.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_propiedades_segmento_2024', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()